# Washington DC Biking data | Hourly Bike Count Prediction

# 3. Modeling for WorkingDays
MBD O-1-5

In [3]:
# To automatically reload the function file 
%load_ext autoreload
%aimport My_Functions
%run My_Functions.py
%autoreload 1
%matplotlib inline
warnings.simplefilter(action='ignore')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
# Load datasets
wd_h=pd.read_csv('./workingdays_data_prepared.csv')
wd_h['dteday']=pd.to_datetime(wd_h['dteday'], format='%Y-%m-%d')

## Subset 2011 -> 2012Q3 Data

#### Dividing the dataset in one year and a half and Q3 iun order to be able to train level 0 AND PREDICT

In [5]:
X_Train_2011_2012Q2 = wd_h[wd_h['dteday'] < datetime.datetime(2012, 7, 5, 0, 0)].drop(['cnt','casual','registered','dteday'],axis=1)## NONE OF THst_EM IN DATA
Y_cnt_train_2011_2012Q2 =wd_h['cnt'][wd_h['dteday'] < datetime.datetime(2012, 7, 5, 0, 0)]

X_Test_2012Q3 = wd_h[(wd_h['dteday'] >= datetime.datetime(2012, 7, 5, 0, 0)) & (wd_h['dteday'] <= datetime.datetime(2012, 9, 30, 0, 0))].drop(['cnt','casual','registered','dteday'],axis=1)## NONE OF THEM IN DATA
Y_cnt_test_2012Q3 =wd_h['cnt'][(wd_h['dteday'] >= datetime.datetime(2012, 7, 5, 0, 0)) & (wd_h['dteday'] <= datetime.datetime(2012, 9, 30, 0, 0))]

## Linear regression w/ GS for *Weekdays* for 2011 -> 2012Q3

Starting with the simplest model on Q3 , even though the score is high here this is the prediction for Q3 (FALL) , as we saw in the EDA FALL SPRING AND SUMMER behaved in the same way but winter was a bit different so we can say that the model learned from 5 similar season and we would not expect the same score for Q4 ( winter ) . This clearly an indicator that the feature created previously have helped our model to learn more about the target

In [10]:
##Linear Regression
lm_parameters = {'fit_intercept':[True,False], 'normalize':[True,False], 'copy_X':[True], "n_jobs": [-1]}

lm = GridSearchCV(LinearRegression(),
                                 param_grid=lm_parameters,
                                 cv=tscv,return_train_score=True)

lm.fit(X_Train_2011_2012Q2, Y_cnt_train_2011_2012Q2)
lm.cv_results_
lm_predictions = lm.predict(X_Test_2012Q3)
lm.score(X_Test_2012Q3, Y_cnt_test_2012Q3)



0.941401943285146

## Random Forest w/ GS for *Weekdays* for 2011 -> 2012Q3

After trying linear regressor we tried random forest with a large grid search in order to make sure that we are using the best possible parameters for the model 
Our expectations were to have a higher score then the linear model but we did not, This could be due to the fact that there are alot of linearly correlated target to the data such as the new weather variable created and the 
mean_per_hour which reflects a direct linear correlation of the past target in the same hour.
Random forest acts best when the data is clearly partitione in groups such as the Rush_Hour and Is_Daylight that clearly differentiate the different target values

In [11]:
RF_parameters = {'n_estimators': [10, 30 ,100],
                                             'bootstrap': [True],
                                             'max_depth': [80, 100 ],
                                             'max_features': ['sqrt',16 ,32],
                                             'min_samples_leaf': [2,  5 , 8],
                                             'min_samples_split': [ 10 , 8 , 15],
                                            'random_state':[random_seed],
                                             "n_jobs": [-1],
                                            'criterion':['mse']}
rf = GridSearchCV(RandomForestRegressor(),
                                 param_grid= RF_parameters,
                                 cv=tscv)

rf.fit(X_Train_2011_2012Q2, Y_cnt_train_2011_2012Q2)
rf.cv_results_
rf_predictions = lm.predict(X_Test_2012Q3)
rf.score(X_Test_2012Q3, Y_cnt_test_2012Q3)


0.9411444427677768

## K Nearest Neighbors regressor w/ GS for *Holidays* for 2011 -> 2012Q3

After trying the basic model we went into the more exotic ones , here the KNN claculate the difference between the differernt point and tries to classsif them 
in groups and then compare these groups to the target and trie to get teh cclosest estimated number.
The bases of trying this model and why it seemed like it would make sense to us is because in our EDA it looked like there are multiple waya to cluster the differnret hours 
either by weather , hr ,season or even day  

In [12]:
knn_parameters = {'n_neighbors':[5,10,15],'weights':['uniform'], 'algorithm':['auto'],'leaf_size':[10, 20,30],
                  'p':[1,2,3],'metric':['minkowski'],"n_jobs": [-1]}
knn = GridSearchCV(KNeighborsRegressor(),
                                 param_grid=knn_parameters,
                                 cv=tscv,return_train_score=True)
knn.fit(X_Train_2011_2012Q2, Y_cnt_train_2011_2012Q2)
# rf.cv_results_
knn_predictions = knn.predict(X_Test_2012Q3)
knn.score(X_Test_2012Q3, Y_cnt_test_2012Q3)

0.9181479043912483

## Support Vector regressor w/ GS for *Holidays* for 2011 -> 2012Q3

This algorithm is known for it complexity and the amount of time it takes oto run but we wanted to give it a try , so instead of using a wide range forthe gridsearch we just used one of the basic kernels  and diffeerent values for C and gamma 
the resulat were Bad , this maybe due to choice of parameters.

In [14]:

svr = GridSearchCV(SVR(kernel='rbf', gamma=0.1), cv=tscv,
                   param_grid={"C": [1e0, 1e1, 1e2, 1e3],
                               "gamma": np.logspace(-2, 2, 5)})

svr.fit(X_Train_2011_2012Q2, Y_cnt_train_2011_2012Q2)
# rf.cv_results_
svr_predictions = svr.predict(X_Test_2012Q3)
svr.score(X_Test_2012Q3, Y_cnt_test_2012Q3)


0.7162265968439802

## XGBoost regressor w/ GS for *Holidays* for 2011 -> 2012Q3

XGBoost is known for winning most of the kaggle competitions so we thought it was worth a shot , specialy since we know that this algorithm work on the errors of a tree algorithm and improves on it
 the Grid Search used wasa wide enough in terms of parameter and gave back the best scores between all the algortihms

In [17]:
param_grid = {'learning_rate': [0.01, 0.1], 
          'max_depth': [4,8,12],
          'min_child_weight': [3,5,10,20,35,50],
          'subsample': [0.5, 0.75],
          'colsample_bytree': [0.5, 0.75],
          'n_estimators': [100, 300], 'random_state':[random_seed]
}

model = xgb.XGBRegressor()

xg = GridSearchCV(model,
                    param_grid = param_grid,
                    cv = tscv,
                    n_jobs = -1,
                    scoring = 'r2',
                    verbose=True)
xg.fit(X_Train_2011_2012Q2, Y_cnt_train_2011_2012Q2)
xg_predictions = xg.predict(X_Test_2012Q3)
xg.score(X_Test_2012Q3, Y_cnt_test_2012Q3)

Fitting 5 folds for each of 288 candidates, totalling 1440 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
exception calling callback for <Future at 0x101e77208 state=finished raised TerminatedWorkerError>
Traceback (most recent call last):
  File "/anaconda3/envs/intro_python/lib/python3.7/site-packages/sklearn/externals/joblib/externals/loky/_base.py", line 625, in _invoke_callbacks
    callback(self)
  File "/anaconda3/envs/intro_python/lib/python3.7/site-packages/sklearn/externals/joblib/parallel.py", line 309, in __call__
    self.parallel.dispatch_next()
  File "/anaconda3/envs/intro_python/lib/python3.7/site-packages/sklearn/externals/joblib/parallel.py", line 731, in dispatch_next
    if not self.dispatch_one_batch(self._original_iterator):
  File "/anaconda3/envs/intro_python/lib/python3.7/site-packages/sklearn/externals/joblib/parallel.py", line 759, in dispatch_one_batch
    self._dispatch(tasks)
  File "/anaconda3/envs/intro_python/lib/python3.7/site-packages/sklearn/externals/joblib/parallel.py", line 7

TerminatedWorkerError: A worker process managed by the executor was unexpectedly terminated. This could be caused by a segmentation fault while calling the function or by an excessive memory usage causing the Operating System to kill the worker. The exit codes of the workers are {SIGABRT(-6), SIGABRT(-6)}

## Taking predictions from LR and Random Forest and xgboost to test on 2012Q4

A level further was taken and the predictions of the top 3 previous algortihms ( Random Forest , Linear Regression , and XGBoost ) on Q3 were attached to the 
initial Q3 data for training a new algorithm on them

In [ ]:
# LR and Random Forest predictions
combinedPredictions = pd.DataFrame({'lm_predictions':lm_predictions,'rf_predictions':rf_predictions,'xg_predictions':xg_predictions })

# Concat original data
X_Test_2012Q3.reset_index(drop=True,inplace=True)
combinedPredictions = pd.concat([combinedPredictions,X_Test_2012Q3], axis=1)

# Target data
combinedPredictionsTarget = pd.DataFrame({'target':Y_cnt_test_2012Q3})

#Reset indices
combinedPredictions.reset_index(drop=True,inplace=True)
combinedPredictionsTarget.reset_index(drop=True,inplace=True)

## Subset Q4 data and concat it with its level-0 predctions

Taking the original Q4 data and adding to it the predictions from the 3 algortihms chosen 

In [ ]:
#Get Q4 data
X_Test_Q4 = wd_h[(wd_h['dteday'] >= datetime.datetime(2012, 10, 1, 0, 0)) & (wd_h['dteday'] <= datetime.datetime(2012, 12, 31, 0, 0))].drop(['cnt','casual','registered','dteday'],axis=1)## NONE OF THEM IN DATA
Y_cnt_test_Q4 =wd_h['cnt'][(wd_h['dteday'] >= datetime.datetime(2012, 10, 1, 0, 0)) & (wd_h['dteday'] <= datetime.datetime(2012, 12, 31, 0, 0))]
oringal_cnt = Y_cnt_test_Q4

#get level-0 predictions for Q4 data
Q4_lm_predictions = pd.DataFrame(lm.predict(X_Test_Q4), columns=["lm_predictions"])
Q4_rf_predictions = pd.DataFrame(rf.predict(X_Test_Q4), columns=["rf_predictions"])
Q4_xg_predictions = pd.DataFrame(xg.predict(X_Test_Q4), columns=["xg_predictions"])

X_Test_Q4.reset_index(drop=True,inplace=True)
Q4_lm_predictions.reset_index(drop=True,inplace=True)
Q4_rf_predictions.reset_index(drop=True,inplace=True)
Q4_xg_predictions.reset_index(drop=True,inplace=True)
Y_cnt_test_Q4.reset_index(drop=True,inplace=True)

#concat Q4 with level-0 predictions 
X_Test_Q4_with_predictions = pd.concat([Q4_lm_predictions, Q4_rf_predictions,Q4_xg_predictions, X_Test_Q4 ], axis=1)
Y_cnt_test_Q4 = pd.DataFrame({"target": Y_cnt_test_Q4})

## Q4 scores

###### Here are the scores of the trained algorithms on the Q4 data in order to be able to compare our initial results to the stacking ones later on

Linear Regression :

In [ ]:
lm.score(X_Test_Q4, Y_cnt_test_Q4)

Random Forest :

In [ ]:
rf.score(X_Test_Q4, Y_cnt_test_Q4)

XGBoost :

In [ ]:
xg.score(X_Test_Q4, Y_cnt_test_Q4)

K Nearest Neighbors:

In [ ]:
knn.score(X_Test_Q4, Y_cnt_test_Q4)

Support Vector Regressor :

In [ ]:
svr.score(X_Test_Q4, Y_cnt_test_Q4)

## Train and test Q4

### XGBoost regressor w/ GS for Level 1 *Holidays* for 2011 -> 2012Q3

According to the scores above  XGBoost will be retrained on the 3rd Quarter with the predictions of the best 3 models and will be tested on Q4  with the predictions on Q4 

In [ ]:
param_grid = {'learning_rate': [0.01, 0.1], 
          'max_depth': [4,8,12],
          'min_child_weight': [3,5,10,20,35,50],
          'subsample': [0.5, 0.75],
          'colsample_bytree': [0.5, 0.75],
           "n_jobs": [-1],
          'n_estimators': [100, 300], 'random_state':[random_seed]
}

model = xgb.XGBRegressor()

xg1 = GridSearchCV(model,
                    param_grid = param_grid,
                    cv = tscv,
                    n_jobs = -1,
                    scoring = 'r2',
                    verbose=True)

xg1.fit(combinedPredictions, combinedPredictionsTarget)
xg1_predictions = xg1.predict(X_Test_Q4_with_predictions)
xg1.score(X_Test_Q4_with_predictions, Y_cnt_test_Q4)

This is the final score for the prediction of Q4 workingdays : 0.9022288970285075

Things that could have increased the score more :
    - CHoosing the right amount of features for every algorithms , but this was very computationaly heavy , since trying an RFECV with the already time ocnsuming algorithms would take too long
    - Increasing the gridsearch or maybe using Bayesian Optimization 
Things we thought would increase our score that we tried but did not work :
    - Running the level 1 algorithm only on the predictions without the initial data 
    - Doing stacking in a way that would use the prediction of the TimeSeriesCross Validation and then retrain the level 0 algorithm with more data 
    (it did not work because the prediction in each fold used was being outputed from a different "Train" of the same algorithm , and the prediction for test data would only be predicted by the training of one of the folds, 
    this means that what the level1 algorithm is trying to learn from the predicted values would be different in each fold )
    

In [ ]:
xg1_predictions  = pd.concat( (pd.DataFrame(xg1.predict(X_Test_Q4_with_predictions)), Y_cnt_test_Q4), axis =1 )
xg1_predictions.to_csv("pred_working.csv", index=False)